# Introduction
This notebook contains the code enabling transformation of an OCEL 2.0 log file to tEKG in live mode.

In [ ]:
import_path= 'shall be set to the /import directory where neo4j can import files'

In [ ]:
import time
import os
from time import gmtime, strftime

In [ ]:
import pm4py
import pandas as pd
import numpy as np
from datetime import datetime
from itertools import product
from neo4j import GraphDatabase

# Setup 
This section sets the initial variables and sets the environment.

In [ ]:
# The notebook is set to transform the running example. 
# To transform other logs, you only need to set the log name properly using the experiment_name variable. It works if the log is in jsonocel format. 
# If your log is in another format, please refine the file_path variable. In this case, you also need to modify the pm4py.read.read_ocel2_json function in the next block to use the correct PM4Py function to read the correct format.

experiment_name = 'runningExample-course'

file_path = './ocel2/'+experiment_name+'.jsonocel'
experiment_path = './experiments/hybrid_'+experiment_name+'.json'

In [ ]:
# legend
# -------------------------------------------------------------------------------------------------
# Any number corresponds to the algorithm 1 line number in the paper
# The section within [] is within the loop given by the number before the open bracket.
# The section within () is within the if statement given by the number before the open parenthesis.

lbl_log = 'LOG'
lbl_class = 'CLASS'
lbl_event = 'EVENT'
lbl_entity =  'ENTITY'
lbl_snapshot = 'SNAPSHOT'
lbl_derived = 'DERIVED'
lbl_has = 'HAS'
lbl_observed = 'OBSERVED'
lbl_rel = 'REL'
lbl_corr='CORR'
lbl_df = 'DF'

lbl_meta_node_log = 'node:Log'  # lines 4-6
lbl_meta_node_class = 'node:Class' # lines 7-9
lbl_meta_node_event = 'node:Event' # lines 10[11-12]
lbl_meta_node_entity = 'node:Entity' # lines 15[16-17]
lbl_meta_node_snapshot = 'node:Snapshot' # lines 15[18,19[20-21]]

# lbl_meta_node_reified = 'node:Reified' # lines 34[35-37] includes the two following
lbl_meta_node_entity_reified = 'node:Reified_Entity' 
lbl_meta_node_snapshot_reified = 'node:Reified_Snapshot'

lbl_meta_rel_log_has_event = 'rel:has' #lines 10[13]
lbl_meta_rel_event_observed_class = 'rel:observed' #lines 10[14]
lbl_meta_rel_entity_snapshot_snapshot = 'rel:snapshot'  # lines 15[18,19[22]]
lbl_meta_rel_snapshot_rel_update_snapshot = 'rel:rel:SnapshotUpdate'  # lines 15[18,23[24(25)]]
lbl_meta_rel_entity_rel_entity = 'rel:rel:Entity' # lines 27[28]
lbl_meta_rel_snapshot_rel_snapshot = 'rel:rel:Snapshot' # lines 27[29[30,31[32(33)]]]

lbl_meta_rel_derived = 'rel:derived' # lines 34[38-39]


lbl_meta_rel_event_corr = 'rel:corr' # 
lbl_meta_rel_event_corr_entity = 'rel:corr:Entity' # lines 40[41]
lbl_meta_rel_event_corr_entity_reified = 'rel:corr:ReifiedEntity' # lines 40[42[43]]

lbl_meta_rel_event_corr_snapshot  = 'rel:corr:Snapshot' # lines 40[44,45[46(47)]]
lbl_meta_rel_event_corr_snapshot_reified  ='rel:corr:ReifiedSnapshot' # lines 40[44,45[46(48[49])]]

lbl_meta_rel_event_df_entity_event  ='rel_Event-df[entity]->Event'
lbl_meta_rel_event_df_snapshot_event  ='rel_Event-df[snapshot]->Event'
lbl_meta_rel_event_df_event='rel:df' # line 50

In [ ]:
meta_time = {}

In [ ]:
URI  = 'bolt://localhost:7687'
AUTH = ('neo4j', '12345678')

In [ ]:
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [ ]:
ocel = pm4py.read.read_ocel2_json(file_path)

In [ ]:
## reset
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query('MATCH (a) DETACH DELETE a')

# Log node
This section adds a node to tEKG for the log.

In [ ]:
action = lbl_meta_node_log
start = time.time()

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query('CREATE (:'+lbl_log+')')

end = time.time()
print(end - start)

meta_time[action] =  end - start

# Class nodes
This section adds nodes to tEKG for classes representing event types in OCEL.

In [ ]:
action = lbl_meta_node_class
start = time.time()

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    for c in ocel.events[ocel.event_activity].unique():
        driver.execute_query("CREATE (:"+lbl_class+" {ID: '"+c+"'})")

end = time.time()
print(end - start)
meta_time[action] =  end - start

# Event nodes
This section adds nodes to tEKG for events corresponding to events in OCEL.

In [ ]:
action = lbl_meta_node_event
start = time.time()

dft = ocel.events.rename(columns={ocel.event_id_column:'EventID', ocel.event_timestamp:'timestamp', ocel.event_activity:'Activity'}).copy()
dft.timestamp = dft.timestamp.dt.strftime('%Y-%m-%dT%H:%M:%S')+'.000+0100'
dft.to_csv(import_path+'events.csv')

cmd = "LOAD CSV  WITH HEADERS FROM 'file:///"+import_path+'events.csv'+"' AS row CREATE (:"+lbl_event+" {EventID: row.EventID, timestamp: datetime(row.timestamp), Activity:row.Activity})"

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(cmd)


end = time.time()
print(end - start)
meta_time[action] =  end - start

In [ ]:
os.remove(import_path+'events.csv')

In [ ]:
cmd =   "CREATE INDEX temp_event_processing IF NOT EXISTS FOR (n:"+lbl_event +") ON (n.EventID)"
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(cmd)

cmd =   "CREATE INDEX temp_event_time_processing IF NOT EXISTS FOR (n:"+lbl_event +") ON (n.timestamp)"
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(cmd)

# Entity nodes
This section adds nodes to tEKG for entities corresponding to objects in OCEL.

In [ ]:
action = lbl_meta_node_entity
start = time.time()

ocel.objects.fillna('').rename(columns={ocel.object_id_column:'ID', ocel.object_type_column:'EntityType'}).to_csv(import_path+'objects.csv')

cols = list(ocel.objects.columns)

for i in range(len(cols)):
    if cols[i] == str(ocel.object_id_column):
        cols[i] = 'ID'
    if cols[i] == str(ocel.object_type_column):
        cols[i] = 'EntityType'

cmd = "LOAD CSV  WITH HEADERS FROM 'file:///"+import_path+'objects.csv'+\
    " ' AS row CREATE (:"+lbl_entity+" { "+\
    ','.join(['`'+c+'`:row.`'+c+'`' for c in cols]) +\
    " }) "

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(cmd)


end = time.time()
print(end - start)
meta_time[action] =  end - start

In [ ]:
os.remove(import_path+'objects.csv')

In [ ]:
cmd =   "CREATE INDEX temp_entity_processing IF NOT EXISTS FOR (n:"+lbl_entity +") ON (n.ID)"
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(cmd)

# Snapshot nodes
This section adds nodes to tEKG for snapshots that materialize OCEL objects when their value has changed.

In [ ]:
action = lbl_meta_node_snapshot
start = time.time()

dft = ocel.objects.copy()
dft['timestamp'] = datetime(1970, 1, 1, 0, 0, 0, tzinfo=pd.Timestamp.utcnow().tzinfo)
dft = dft.rename(columns={ocel.object_id_column: "ID", ocel.object_type_column: "EntityType"}, errors="raise")
dft2 = ocel.object_changes.rename(columns={ocel.object_id_column: "ID", ocel.object_type_column: "EntityType", ocel.event_timestamp: "timestamp"}, errors="raise").copy()

dft = pd.concat([dft, dft2]).sort_values(['ID', 'timestamp'])
dft = dft.groupby('ID').apply(lambda group: group.ffill())
dft = dft.drop(columns=[ocel.changed_field])
dft['ENTITY_ID'] = dft['ID']
dft['ID'] =  '(' + dft['ID'] + ',' + dft['timestamp'].astype('str') + ')' 
dft = dft.reset_index(drop=True)

dft.timestamp = dft.timestamp.dt.strftime('%Y-%m-%dT%H:%M:%S')+'.000+0100'

dft.to_csv(import_path+'snapshots.csv')

cols = list(dft.columns)


cmd = "LOAD CSV  WITH HEADERS FROM 'file:///"+import_path+'snapshots.csv'+\
    " ' AS row CREATE (:"+lbl_snapshot+" { "+\
    ','.join(['`'+c+'`:row.`'+c+'`' for c in cols if c!='timestamp' ]) +\
    " , timestamp:datetime(row.timestamp)}) "


with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(cmd)


end = time.time()
print(end - start)
meta_time[action] =  end - start

In [ ]:
os.remove(import_path+'snapshots.csv')

In [ ]:
# cmd =   "CREATE INDEX temp_snapshot_processing_entity_id IF NOT EXISTS FOR (n:"+lbl_snapshot +") ON (n.ENTITY_ID)"
# with GraphDatabase.driver(URI, auth=AUTH) as driver:
#     driver.execute_query(cmd)

cmd =   "CREATE INDEX temp_snapshot_timestamp_processing IF NOT EXISTS FOR (n:"+lbl_snapshot +") ON (n.timestamp)"
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(cmd)

cmd =   "CREATE INDEX temp_snapshot_processing IF NOT EXISTS FOR (n:"+lbl_snapshot +") ON (n.ID)"
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(cmd)

# has edges
This section adds edges labeled "has" to tEKG to connect the Log node to the Event nodes.

In [ ]:
action = lbl_meta_rel_log_has_event
start = time.time()

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query('MATCH (l:'+lbl_log+'), (e:'+lbl_event+') MERGE (l)-[:'+lbl_has+']->(e)')

end = time.time()
print(end - start)
meta_time[action] =  end - start

# observed edges
This section adds edges labeled "observed" to tEKG to connect Event nodes to Class nodes.

In [ ]:
action = lbl_meta_rel_event_observed_class
start = time.time()

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query('MATCH (e:'+lbl_event+'), (c:'+lbl_class+') WHERE  e.Activity=c.ID MERGE (e)-[:'+lbl_observed+']->(c)')

end = time.time()
print(end - start)
meta_time[action] =  end - start

# rel edges (Entity2Entity)
This section adds edges labeled "rel" to tEKG to connect Entity nodes to Entity nodes.

In [ ]:
action = lbl_meta_rel_entity_rel_entity
start = time.time()

ocel.o2o.rename(columns={ocel.object_id_column: "ID", ocel.object_id_column+'_2': "ID2", ocel.qualifier: "qual"}, errors="raise").to_csv(import_path+'o2o.csv')

cmd = "LOAD CSV  WITH HEADERS FROM 'file:///"+import_path+'o2o.csv'+\
    " ' AS row CREATE (:O2O { ID: row.ID, ID2: row.ID2, qual:row.qual }) "
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(cmd)

print('O2O data is loaded - time: ', strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime()))

cmd =   "MATCH (o:O2O), (a:"+lbl_entity +"  {ID:o.ID}), (b:"+lbl_entity +" {ID:o.ID2}) MERGE (a)-[:"+lbl_rel +" {qual:o.qual}]->(b)" 
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(cmd)


cmd =   "MATCH (o:O2O) DELETE o" 
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(cmd)
    
end = time.time()
print(end - start)
meta_time[action] =  end - start

In [ ]:
os.remove(import_path+'o2o.csv')

# snapshot edges
This section adds edges labeled "snapshot" to tEKG to connect Entity nodes to Snapshot nodes.

In [ ]:
action = lbl_meta_rel_entity_snapshot_snapshot
start = time.time()

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query('MATCH (o1:'+lbl_entity+'), (o2:'+lbl_snapshot+' {ENTITY_ID: o1.ID}) MERGE (o1)-[:'+lbl_snapshot+']->(o2)')

end = time.time()
print(end - start)
meta_time[action] =  end - start

# rel edges (update)
This section adds edges labeled "rel" to tEKG to connect Snapshot nodes to Snapshot nodes. These edges show the lifecycle of an object through time, during which the value of the object's properties changed over time. 

In [ ]:
action = lbl_meta_rel_snapshot_rel_update_snapshot
start = time.time()


with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(""+
        "MATCH (n:"+lbl_entity+")-[:"+lbl_snapshot+"]->(i:"+lbl_snapshot+") "+
        "WITH n, i order by i.timestamp "+
        "WITH n, collect(i) as iss, range(0, size(collect(i))-2) as issn "+
        "UNWIND issn as i MATCH (a), (b) "+
        "WHERE a=iss[i] and b=iss[i+1] "+
        "MERGE (a)-[:"+lbl_rel+" {qual:'UPDATE'}]->(b)")

end = time.time()
print(end - start)
meta_time[action] =  end - start

# rel edges (Snapshot2Snapshot)
This section adds edges labeled "rel" to tEKG to connect Snapshot nodes to Snapshot nodes.

In [ ]:
action = lbl_meta_rel_snapshot_rel_snapshot
start = time.time()

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(""+
            "MATCH (e1i:"+lbl_snapshot+")<-[:"+lbl_snapshot+"]-(e1:"+lbl_entity+")-[r:"+lbl_rel+"]->(e2:"+lbl_entity+")-[:"+lbl_snapshot+"]->(e2i:"+lbl_snapshot+") "+
            "WHERE e1i.timestamp>=e2i.timestamp "+
            "WITH e1i, r.qual as qualification, e2i order by e2i.timestamp desc "+
            "WITH e1i, collect(e2i)[0] as e2i, qualification "+
            "MATCH (a), (b) "+
            "WHERE a=e1i and b=e2i "+
            "MERGE  (a)-[:"+lbl_rel+" {qual:qualification}]->(b) ")

end = time.time()
print(end - start)
meta_time[action] =  end - start

# Entity nodes (reified)
This section adds nodes labeled "Entity" to tEKG for the reified ones.

In [ ]:
action = lbl_meta_node_entity_reified 
start = time.time()

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(
            "MATCH (a:"+lbl_entity+")-[r:REL]->(b:"+lbl_entity+") "+
            "CREATE (c:"+lbl_entity+":REIFIED {ID:'('+a.ID+','+b.ID+')', EntityType:'('+a.EntityType+','+b.EntityType+')', rel1:id(a), rel2:id(b)})"
        )
    
end = time.time()
print(end - start)
meta_time[action] =  end - start

# Snapshot nodes (reified)
This section adds nodes labeled "Snapshot" to tEKG for the reified ones.

In [ ]:
action = lbl_meta_node_snapshot_reified 
start = time.time()

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    
    driver.execute_query(
            "MATCH (a:"+lbl_snapshot+")-[r:REL]->(b:"+lbl_snapshot+") "+
            "CREATE (c:"+lbl_snapshot+":REIFIED {ID:'('+a.ID+','+b.ID+')', EntityType:'('+a.EntityType+','+b.EntityType+')', rel1:id(a), rel2:id(b)})"
        )
    
end = time.time()
print(end - start)
meta_time[action] =  end - start

# derived edges
This section adds edges labeled "derived" to tEKG to connect reified nodes (either Snapshot or Entity) to their corresponding Snapshot or Entity nodes.

In [ ]:
action = lbl_meta_rel_derived
start = time.time()

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    cmds = {
        0: "MATCH (b:"+lbl_entity+") SET b.sysid=id(b)" ,
        1: "MATCH (b:"+lbl_snapshot+") SET b.sysid=id(b)",
        2: "CREATE INDEX temp_sysidfor_entity IF NOT EXISTS FOR (n:"+lbl_entity +") ON (n.sysid)",
        3: "CREATE INDEX temp_sysidfor_snapshot IF NOT EXISTS FOR (n:"+lbl_snapshot +") ON (n.sysid)",
    }
    for idx, cmd in cmds.items():
        driver.execute_query(cmd)
        

    print('prelinimaries are set - time: ', strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime()))

    driver.execute_query(
            "MATCH (a:REIFIED), (b:"+lbl_entity+") "+
            "WHERE b.sysid=a.rel1 or b.sysid=a.rel2 "+
            "MERGE (a)-[:"+lbl_derived+"]->(b)"
        )

    print('reified entities are updated - time: ', strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime()))
    
    driver.execute_query(
            "MATCH (a:REIFIED), (b:"+lbl_snapshot+") "+
            "WHERE b.sysid=a.rel1 or b.sysid=a.rel2 "+
            "MERGE (a)-[:"+lbl_derived+"]->(b)"
        )

    print('reified snapshots are updated - time: ', strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime()))

    driver.execute_query(
            "MATCH (n:REIFIED) "+
            "REMOVE n.rel1, n.rel2"
        )

    print('rel 1 & 2 are deleted- time: ', strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime()))

    driver.execute_query(
            "MATCH (n:REIFIED) "+
            "REMOVE n:REIFIED"
        )

    print('reified nodes are deleted - time: ', strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime()))
    
    driver.execute_query(
            "MATCH (a)<-[:"+lbl_derived+"]-(x)-[:"+lbl_derived+"]->(b) "+
            "WHERE a.timestamp IS NOT NULL and a.timestamp<=b.timestamp "+ # is not null is replaced with exist() function in newer neo4j versions.
            "SET "+
            "    x.root1ID = a.ENTITY_ID, "+ #rootID
            "    x.root2ID = b.ENTITY_ID, "+ #rootID
            "    x.root1timestamp = a.timestamp, "+
            "    x.root2timestamp = b.timestamp"
        )

    print('ready for clean up - time: ', strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime()))
    
    cmds = {
        0: "DROP INDEX temp_sysidfor_entity",
        1: "DROP INDEX temp_sysidfor_snapshot",
    }
    for idx, cmd in cmds.items():
        driver.execute_query(cmd)
        
end = time.time()
print(end - start)
meta_time[action] =  end - start

# corr edges
This section adds edges labeled "corr" to tEKG. 

In [ ]:
action = lbl_meta_rel_event_corr
start = time.time()

ocel.relations.rename(columns={
    ocel.event_id_column: "EventID",
    ocel.event_activity:'Activity',
    ocel.event_timestamp:'timestamp',
    ocel.object_id_column: "ObjectID", 
    ocel.object_type_column:'EntityType',
    ocel.qualifier: "qual"}, errors="raise").to_csv(import_path+'corr.csv')

cmd = "LOAD CSV  WITH HEADERS FROM 'file:///"+import_path+'corr.csv'+\
    " ' AS row CREATE (:E2O { EventID: row.EventID, Activity: row.Activity, timestamp:row.timestamp, ObjectID:row.ObjectID , EntityType:row.EntityType }) "
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(cmd)

print('E2O data is loaded - time: ', strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime()))
    
# corr entities
cmd =   "MATCH (o:E2O),(a:"+lbl_event +"  {EventID:o.EventID}),(b:"+lbl_entity +" {ID:o.ObjectID}) MERGE (a)-[:"+lbl_corr +"]->(b)" 
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(cmd)

print('corr for entities are loaded - time: ', strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime()))

# corr snapshots
cmd =   "MATCH (o2o:E2O),(e:"+lbl_event +"),(o:"+lbl_snapshot +") " +\
            "WHERE e.timestamp>=o.timestamp AND o2o.EventID=e.EventID AND o2o.ObjectID=o.ENTITY_ID " + \
            "WITH e, o order by o.timestamp desc " + \
            "WITH e, o.EntityType as et, collect(o)[0] as o " + \
            "MERGE (e)-[:"+lbl_corr+"]->(o)"
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(cmd)

print('corr for snapshots are loaded - time: ', strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime()))

# corr reified entities
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(
            "MATCH (e:"+lbl_event+")-[:"+lbl_corr+"]->(o:"+lbl_entity+") "+
            "MATCH (o1)-[:"+lbl_derived+"]->(o) "+
            "MERGE (e)-[:"+lbl_corr+"]->(o1) "
        )

print('corr for reified entities are loaded - time: ', strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime()))

# corr reified snapshots
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(
            "MATCH (e:"+lbl_event+")-[:"+lbl_corr+"]->(o:"+lbl_snapshot+") "+
            "MATCH (o1)-[:"+lbl_derived+"]->(o) "+
            "MERGE (e)-[:"+lbl_corr+"]->(o1) "
        )

print('corr for reified snapshots are loaded - time: ', strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime()))

end = time.time()
print(end - start)
meta_time[action] =  end - start

In [ ]:
cmd =   "MATCH (o:O2O) DELETE o" 
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(cmd)

In [ ]:
# drop temps
cmd =   "MATCH (o2o:E2O) DELETE o2o"
                
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(cmd)

In [ ]:
os.remove(import_path+'corr.csv')

# df edges
This section adds edges labeled "df" to tEKG to connect Events nodes to Entity or Snapshot nodes.

In [ ]:
action = lbl_meta_rel_event_df_event
start = time.time()

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    
    driver.execute_query(
    "MATCH (e:"+lbl_event+")-[:"+lbl_corr+"]->(n)  " +
    "WITH n, e order by e.timestamp  " +
    "WITH n, collect(e) as es, range(0, size(collect(e))-2) as esn " +
    "UNWIND esn as i  " +
    "MATCH (a:"+lbl_event+"), (b:"+lbl_event+")  " +
    "WHERE a=es[i] and b=es[i+1]  " +
    "MERGE (a)-[:"+lbl_df+" {EntityType:n.EntityType, EntityID:n.ID}]->(b) "
    )

print('df are added - time: ', strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime()))

# removing parallel dfs

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(
        "MATCH ()-[r:"+lbl_df+"]->() " +
        "SET r.addNewKnowledge = TRUE " 
    )
    driver.execute_query(
        "MATCH (n)<-[:"+lbl_corr+"]-(e1:"+lbl_event+")-[r1:"+lbl_df+" {EntityID:n.ID}]->(e2:"+lbl_event+") " +
        "MATCH (n)<-[:"+lbl_derived+"]-(rn)<--(e1)-[r2:"+lbl_df+"  {EntityID:rn.ID}]->(e2) " +
        "SET r2.addNewKnowledge = FALSE " 
    )
    driver.execute_query(
        "MATCH ()-[rb:"+lbl_df+" {addNewKnowledge:TRUE}]->()-[r:"+lbl_df+"  {EntityID:rb.ID}]->()-[ra:"+lbl_df+"  {EntityID:r.ID, addNewKnowledge:TRUE}]->() " +
        "SET r.addNewKnowledge=TRUE " 
    )
    driver.execute_query(
        "MATCH ()-[r:"+lbl_df+" {addNewKnowledge:FALSE}]->() " +
        "DELETE r " 
    )
    driver.execute_query(
        "MATCH ()-[r:"+lbl_df+"]->() " +
        "REMOVE  r.addNewKnowledge " 
    )


end = time.time()
print(end - start)
meta_time[action] =  end - start

# Conclude

In [ ]:
cmd =   "DROP INDEX temp_entity_processing"
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(cmd)

cmd =   "DROP INDEX temp_snapshot_timestamp_processing"
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(cmd)

cmd =   "DROP INDEX temp_snapshot_processing"
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(cmd)

cmd =   "DROP INDEX temp_event_processing"
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(cmd)

cmd =   "DROP INDEX temp_event_time_processing"
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.execute_query(cmd)

# Exporting meta data

In [ ]:
import json

with open(experiment_path, "w") as fp:
    json.dump(meta_time, fp)  # encode dict into JSON